In [8]:
import networkx as nx
import math

In [9]:
class WRNEL:
    '''
    Paper : Study on dynamic evolution characteristics of Wuhan metro network based on complex network, 
    Physica A: Statistical Mechanics and Its Applications 648 (2024) 129945. https://doi.org/10.1016/j.physa.2024.129945.
    author: Kangzheng Huang
    '''

    def __init__(self, G):
    
        '''
        initialisation
        '''

        self.G = G.copy()
        self.nodes = list(self.G.nodes())
        self.degs  = dict(nx.degree(self.G))
        self.edges = list(self.G.edges())
        self.nbrs  = {}
        self.si = {}


        for node in self.nodes:
            nbrs = list(nx.neighbors(self.G, node))
            self.nbrs[node] = nbrs
            self.si[node] = 0
        
        for u,v,d in self.G.edges(data=True):
            self.si[u] += d['weight']
            self.si[v] += d['weight']


    def cal_lj(self):
        '''
        Eq. 40, 25
        '''
        self.lj = {}
        for node in self.nodes:
            self.lj[node] = 0
            for nodej in self.nodes:
                if node != nodej:
                    try:
                        dij = nx.shortest_path_length(self.G, node, nodej, weight='weight') # Eq. 49
                        self.lj[node] += self.degs[node] / (dij)**12 - self.degs[nodej] / (dij)**6
                    except:
                        self.lj[node] += 0

    def cal_sim(self,listb,listc):
        '''
        Eq. 6
        '''
        resb = []
        resa = []
        resb.extend(listb)
        resb.extend(listc)
        resb = list(set(resb))
        for i in resb:
            if i in listb and i in listc:
                resa.append(i)
        return len(resa) / len(resb)
    
    def cal_lls(self):
        '''
        local link similarity
        '''
        self.lls = {}
        for node in self.nodes:
            nbrs = self.nbrs[node]
            if len(nbrs) <= 1:
                self.lls[node] = 0
            else:
                T = 0
                for i in range(len(nbrs)-1):
                    for j in range(i+1, len(nbrs)):
                        if (nbrs[i],nbrs[j]) in self.edges or (nbrs[j],nbrs[i]) in self.edges:
                            T += 0
                        else:
                            nodea = nbrs[i]
                            nodeb = nbrs[j]
                            listb = self.nbrs[nodeb]
                            listc = self.nbrs[nodea]
                            T += 1 - self.cal_sim(listb,listc) # Eq. 7
                self.lls[node] = T


    def cal_entropy(self):
        '''
        Eq. 50, 27
        '''
        self.Ev = {}
        for node in self.nodes:
            self.Ev[node] = 0
            nbrs = list(self.G.neighbors(node))

            for nbr in nbrs:
                nbrs1 = list(self.G.neighbors(node))
                nbr_all_degree = 0
                for nbr1 in nbrs1:
                    nbr_all_degree += self.si[nbr1]

                puv = self.si[nbr] / nbr_all_degree # Eq. 50
                self.Ev[node] += - (puv * math.log(puv, 2) + (1-puv) * math.log((1-puv+0.0001),2))

    def main(self):

        self.cal_lj()
        self.cal_lls()
        self.cal_entropy()

        self.rlls = {} # Eq. 31
        self.rni  = {} # Eq. 29
        self.rev  = {} # Eq. 30

        for node in self.nodes:

            self.rlls[node] = 0
            self.rni[node] = 0
            self.rev[node] = 0

            nbrs = self.nbrs[node]
            for nbr in nbrs:

                if self.lls[node] - self.lls[nbr] > 0:
                    self.rlls[node] += self.lls[node] - self.lls[nbr]

                if self.lj[node] - self.lj[nbr] > 0:
                    self.rni[node] += self.lj[node] - self.lj[nbr]

                if self.Ev[node] - self.Ev[nbr] > 0:
                    self.rev[node] += self.Ev[node] - self.Ev[nbr]


        all_nums_max = max(list([v for k,v in self.lls.items()])) 
        all_nums_min = min(list([v for k,v in self.lls.items()]))
        all_nums_max1 = max(list([v for k,v in self.lj.items()])) 
        all_nums_min1 = min(list([v for k,v in self.lj.items()]))
        all_nums_max2 = max(list([v for k,v in self.Ev.items()])) 
        all_nums_min2 = min(list([v for k,v in self.Ev.items()])) 
        
        self.nel = {} # Eq. 36 
        for node in self.nodes:
            
            # Eq. 33-35
            try:
                a =  (self.rlls[node] - all_nums_min) / (all_nums_max - all_nums_min)
            except:
                a = 0
            try:
                b = (self.rni[node] - all_nums_min1) / (all_nums_max1 - all_nums_min1)
            except:
                b = 0
            try:
                c = (self.rev[node] - all_nums_min2) / (all_nums_max2 - all_nums_min2)
            except:
                c = 0

            self.nel[node] = a + b + c

        # Eq. 37
        self.rnel = {}
        for node in self.nodes:
            self.rnel[node] = 0
            nbrs = self.nbrs[node]
            for nbr in nbrs:
                if self.nel[node] - self.nel[nbr] > 0:
                    self.rnel[node] += self.nel[node] - self.nel[nbr]

        return self.rnel

    

In [10]:
G = nx.karate_club_graph()
edges = list(G.edges())
G1 = nx.Graph()
for edge in edges:
    G.add_edge(edge[0], edge[1], weight=1)

rnel = WRNEL(G).main()